# Results p. 3
## Fixation Discriminability
### How sensitive are detectors to fixation onsets/offsets?

In [28]:
import os
import copy

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import peyes

from analysis._article_results.lund2013._helpers import *
import analysis.statistics.channel_sdt as ch_sdt

# pio.renderers.default = "browser"
FIG_ID, IS_SUPP = 5, False

## Load Data

In [29]:
LABEL = 1       # EventLabelEnum.FIXATION.value
THRESHOLD = 5   # samples

csdt_metrics = ch_sdt.load(
    dataset_name=DATASET_NAME,
    output_dir=PROCESSED_DATA_DIR,
    label=LABEL,
    stimulus_type=STIMULUS_TYPE,
    channel_type=None,
)
csdt_metrics.drop(index=['P', 'PP', 'N', 'TP'], level=peyes.constants.METRIC_STR, inplace=True)    # Remove unused metrics

csdt_metrics

trial_id                                       25                      \
gt                                             RA                       
pred                                           MN   engbert  remodnav   
channel_type metric           threshold                                 
onset        recall           0          0.285714  0.107143  0.071429   
             precision        0          0.296296  0.090909  0.166667   
             f1               0          0.290909  0.098361  0.100000   
             false_alarm_rate 0          0.003831  0.006048  0.002016   
             d_prime          0          2.100695  1.267442  1.410393   
...                                           ...       ...       ...   
offset       precision        20         0.962963  0.848485  0.333333   
             f1               20         0.945455  0.918033  0.200000   
             false_alarm_rate 20         0.010677  0.053385  0.085417   
             d_prime          20         3.766897  3.954658  0.301961   
             criterion        20         0.418215 -0.427646  1.218551   

trial_id                                                               \
gt                                                                      
pred                                         idvt        nh       idt   
channel_type metric           threshold                                 
onset        recall           0          0.035714  0.071429  0.035714   
             precision        0          0.055556  0.074074  0.055556   
             f1               0          0.043478  0.072727  0.043478   
             false_alarm_rate 0          0.003427  0.005040  0.003427   
             d_prime          0          0.901072  1.107817  0.901072   
...                                           ...       ...       ...   
offset       precision        20         0.944444  0.962963  0.944444   
             f1               20         0.739130  0.945455  0.739130   
             false_alarm_rate 20         0.010677  0.010677  0.010677   
             d_prime          20         2.573543  3.766897  2.573543   
             criterion        20         1.014892  0.418215  1.014892   

trial_id                                                               \
gt                                                                 MN   
pred                                         ivvt       ivt        RA   
channel_type metric           threshold                                 
onset        recall           0          0.178571  0.035714  0.296296   
             precision        0          0.147059  0.030303  0.285714   
             f1               0          0.161290  0.032787  0.290909   
             false_alarm_rate 0          0.005847  0.006452  0.004031   
             d_prime          0          1.600437  0.683686  2.114342   
...                                           ...       ...       ...   
offset       precision        20         0.794118  0.787879  0.928571   
             f1               20         0.870968  0.852459  0.945455   
             false_alarm_rate 20         0.074740  0.074740  0.021129   
             d_prime          20         3.244117  2.906607  3.817134   
             criterion        20        -0.180685 -0.011930  0.122412   

trial_id                                           ...        44            \
gt                                                 ...        RA             
pred                                      engbert  ...      ivvt       ivt   
channel_type metric           threshold            ...                       
onset        recall           0          0.148148  ...  0.000000  0.000000   
             precision        0          0.121212  ...  0.000000  0.000000   
             f1               0          0.133333  ...       NaN       NaN   
             false_alarm_rate 0          0.005846  ...  0.015244  0.013211   
             d_prime          0          1.476922  ... -1.139018 -1.084435   
...       

## Onset Detection

In [30]:
onset_statistics, onset_pvalues, onset_nemenyi, onset_Ns = ch_sdt.friedman_nemenyi(
    csdt_metrics, "onset", THRESHOLD, [GT1, GT2]
)

onset_pvalues <= ALPHA

gt,MN,RA
metric,,
criterion,True,True
d_prime,True,True
f1,True,True
false_alarm_rate,True,True
precision,True,True
recall,True,True


In [31]:
pd.concat([onset_statistics, onset_pvalues], axis=1, keys=['Q', 'p']).stack(1, future_stack=True)

Q             p
metric           gt                         
criterion        MN  65.783226  2.984602e-12
                 RA  93.570652  5.484541e-18
d_prime          MN  69.530323  5.104016e-13
                 RA  90.157609  2.809965e-17
f1               MN  69.158710  6.082556e-13
                 RA  71.474359  2.037246e-13
false_alarm_rate MN  44.961039  4.764385e-08
                 RA  55.673507  3.388724e-10
precision        MN  49.616580  5.610951e-09
                 RA  68.967213  6.657793e-13
recall           MN  73.873016  6.547338e-14
                 RA  93.607843  5.387685e-18

### Post Hoc Analysis
#### $d'$

In [32]:
post_hoc_onset_dprime = ch_sdt.post_hoc_table(
    onset_nemenyi, peyes.constants.D_PRIME_STR, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)
post_hoc_onset_dprime

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *       *    n.s.    n.s.     n.s.
         RA      --    n.s.     ***      **    n.s.    n.s.     n.s.
ivvt     MN  0.9467      --     ***     ***    n.s.    n.s.     n.s.
         RA  0.9590      --     ***     ***    n.s.    n.s.     n.s.
idt      MN  0.0262  0.0002      --    n.s.     ***      **     n.s.
         RA  0.0010  0.0000      --    n.s.     ***     ***     n.s.
idvt     MN  0.0322  0.0003  1.0000      --     ***      **     n.s.
         RA  0.0012  0.0000  1.0000      --     ***     ***     n.s.
engbert  MN  0.5759  0.9926  0.0000  0.0000      --    n.s.        *
         RA  0.5373  0.9841  0.0000  0.0000      --    n.s.        *
nh       MN  0.9986  0.9987  0.0032  0.0041  0.8863      --     n.s.
         RA  1.0000  0.9919  0.0002  0.0003  0.7200      --     n.s.
remodnav MN  0.9046  0.2653  0.4979  0.5397  0.0383  0.6100       --
         RA  0.8462  0.2215  0.1394  0.1546  0.0193  0.6924       --

#### $f1$

In [33]:
post_hoc_onset_f1 = ch_sdt.post_hoc_table(
    onset_nemenyi, peyes.constants.F1_STR, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)
post_hoc_onset_f1

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.      **      **    n.s.    n.s.     n.s.
         RA      --    n.s.      **      **    n.s.    n.s.     n.s.
ivvt     MN  0.9970      --     ***     ***    n.s.    n.s.     n.s.
         RA  0.9944      --     ***     ***    n.s.    n.s.     n.s.
idt      MN  0.0061  0.0003      --    n.s.     ***      **     n.s.
         RA  0.0016  0.0001      --    n.s.     ***     ***     n.s.
idvt     MN  0.0078  0.0005  1.0000      --     ***      **     n.s.
         RA  0.0018  0.0001  1.0000      --     ***      **     n.s.
engbert  MN  0.8120  0.9873  0.0000  0.0000      --    n.s.        *
         RA  0.7876  0.9897  0.0000  0.0000      --    n.s.        *
nh       MN  1.0000  0.9999  0.0018  0.0024  0.9269      --     n.s.
         RA  1.0000  0.9978  0.0010  0.0011  0.8424      --     n.s.
remodnav MN  0.7271  0.3178  0.4827  0.5244  0.0404  0.5397       --
         RA  0.6217  0.1948  0.3419  0.3576  0.0199  0.5484       --

#### Criterion

In [34]:
post_hoc_onset_crit = ch_sdt.post_hoc_table(
    onset_nemenyi, peyes.constants.CRITERION_STR, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)
post_hoc_onset_crit

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *       *    n.s.    n.s.     n.s.
         RA      --    n.s.     ***     ***    n.s.    n.s.     n.s.
ivvt     MN  0.5873      --     ***     ***    n.s.       †      ***
         RA  0.6119      --     ***     ***    n.s.       *      ***
idt      MN  0.0247  0.0000      --    n.s.     ***    n.s.     n.s.
         RA  0.0008  0.0000      --    n.s.     ***    n.s.     n.s.
idvt     MN  0.0181  0.0000  1.0000      --     ***    n.s.     n.s.
         RA  0.0006  0.0000  1.0000      --     ***       †     n.s.
engbert  MN  0.7807  1.0000  0.0000  0.0000      --    n.s.       **
         RA  0.7840  1.0000  0.0000  0.0000      --    n.s.      ***
nh       MN  0.9369  0.0562  0.4161  0.3595  0.1315      --     n.s.
         RA  0.9116  0.0484  0.0805  0.0699  0.1071      --     n.s.
remodnav MN  0.3114  0.0007  0.9717  0.9559  0.0029  0.9412       --
         RA  0.0906  0.0001  0.8968  0.8780  0.0002  0.7486       --

## Offset Detection

In [35]:
offset_statistics, offset_pvalues, offset_nemenyi, offset_Ns = ch_sdt.friedman_nemenyi(
    csdt_metrics, "offset", THRESHOLD, [GT1, GT2]
)

offset_pvalues <= ALPHA

gt,MN,RA
metric,,
criterion,True,True
d_prime,True,True
f1,True,True
false_alarm_rate,True,True
precision,True,True
recall,True,True


In [36]:
pd.concat([offset_statistics, offset_pvalues], axis=1, keys=['Q', 'p']).stack(1, future_stack=True)

Q             p
metric           gt                          
criterion        MN   68.945736  6.725602e-13
                 RA   98.235721  5.854802e-19
d_prime          MN   72.000000  1.588873e-13
                 RA   94.242974  3.974055e-18
f1               MN   58.977376  7.260057e-11
                 RA   79.778226  3.970301e-15
false_alarm_rate MN   33.683089  7.744746e-06
                 RA   53.316033  1.013328e-09
precision        MN   57.643979  1.353008e-10
                 RA   68.360476  8.863997e-13
recall           MN   73.435484  8.054783e-14
                 RA  102.486334  7.595625e-20

### Post Hoc Analysis
#### $d'$

In [37]:
post_hoc_offset_dprime = ch_sdt.post_hoc_table(
    offset_nemenyi, peyes.constants.D_PRIME_STR, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)
post_hoc_offset_dprime

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.      **      ***
         RA      --    n.s.       *       *    n.s.     ***      ***
ivvt     MN  1.0000      --    n.s.    n.s.    n.s.      **      ***
         RA  0.9999      --    n.s.    n.s.    n.s.      **      ***
idt      MN  0.0925  0.1755      --    n.s.       *    n.s.     n.s.
         RA  0.0275  0.0850      --    n.s.      **    n.s.     n.s.
idvt     MN  0.1333  0.2382  1.0000      --       *    n.s.     n.s.
         RA  0.0393  0.1138  1.0000      --      **    n.s.        †
engbert  MN  0.9998  0.9967  0.0285  0.0450      --     ***      ***
         RA  0.9907  0.9338  0.0012  0.0020      --     ***      ***
nh       MN  0.0023  0.0067  0.9592  0.9252  0.0004      --     n.s.
         RA  0.0004  0.0024  0.9670  0.9451  0.0000      --     n.s.
remodnav MN  0.0000  0.0000  0.2410  0.1778  0.0000  0.8634       --
         RA  0.0000  0.0000  0.0846  0.0620  0.0000  0.5875       --

#### $f1$

In [38]:
post_hoc_offset_f1 = ch_sdt.post_hoc_table(
    offset_nemenyi, peyes.constants.F1_STR, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)
post_hoc_offset_f1

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.       *      ***
         RA      --    n.s.       *       †    n.s.      **      ***
ivvt     MN  0.9541      --    n.s.    n.s.    n.s.    n.s.       **
         RA  0.9220      --    n.s.    n.s.    n.s.    n.s.      ***
idt      MN  0.0837  0.6304      --    n.s.    n.s.    n.s.     n.s.
         RA  0.0430  0.5640      --    n.s.       *    n.s.     n.s.
idvt     MN  0.1107  0.6976  1.0000      --    n.s.    n.s.     n.s.
         RA  0.0527  0.6085  1.0000      --       *    n.s.     n.s.
engbert  MN  1.0000  0.9629  0.0946  0.1242      --       *      ***
         RA  1.0000  0.8608  0.0249  0.0310      --     ***      ***
nh       MN  0.0113  0.2479  0.9976  0.9942  0.0134      --     n.s.
         RA  0.0014  0.1025  0.9814  0.9731  0.0006      --     n.s.
remodnav MN  0.0000  0.0012  0.3035  0.2501  0.0000  0.6827       --
         RA  0.0000  0.0001  0.1071  0.0902  0.0000  0.5593       --

#### Criterion

In [39]:
post_hoc_offset_crit = ch_sdt.post_hoc_table(
    offset_nemenyi, peyes.constants.CRITERION_STR, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)
post_hoc_offset_crit

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *       *    n.s.    n.s.      ***
         RA      --    n.s.      **      **    n.s.    n.s.      ***
ivvt     MN  0.9709      --      **     ***    n.s.       *      ***
         RA  0.9909      --     ***     ***    n.s.       *      ***
idt      MN  0.0490  0.0012      --    n.s.      **    n.s.     n.s.
         RA  0.0017  0.0000      --    n.s.     ***    n.s.     n.s.
idvt     MN  0.0254  0.0004  1.0000      --      **    n.s.     n.s.
         RA  0.0010  0.0000  1.0000      --     ***    n.s.     n.s.
engbert  MN  0.9961  1.0000  0.0044  0.0019      --       †      ***
         RA  0.9762  1.0000  0.0000  0.0000      --      **      ***
nh       MN  0.3098  0.0240  0.9924  0.9733  0.0637      --     n.s.
         RA  0.1469  0.0137  0.8895  0.8409  0.0076      --     n.s.
remodnav MN  0.0002  0.0000  0.8646  0.9347  0.0000  0.4070       --
         RA  0.0000  0.0000  0.8858  0.9249  0.0000  0.1577       --

## Figures
### (1) Onset Distributions

In [40]:
onset_distribution_figure = ch_sdt.single_threshold_figure(
    csdt_metrics.loc[(slice(None), [peyes.constants.D_PRIME_STR, peyes.constants.F1_STR], slice(None)), :],
    peyes.constants.ONSET_STR,
    THRESHOLD,
    GT1,
    gt2=GT2,
    show_other_gt=True,
    share_x=True,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
)

onset_distribution_figure.update_layout(
    title=dict(text="fixation onsets"),
    width=1000, height=500,
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=10, r=10, b=10, t=10, pad=0),
)
# onset_distribution_figure.layout.annotations = []   # remove subtitles

onset_distribution_figure.show()

### (2) Offset Distributions

In [41]:
offset_distribution_figure = ch_sdt.single_threshold_figure(
    csdt_metrics.loc[(slice(None), [peyes.constants.D_PRIME_STR, peyes.constants.F1_STR], slice(None)), :],
    peyes.constants.OFFSET_STR,
    THRESHOLD,
    GT1,
    gt2=GT2,
    show_other_gt=True,
    share_x=True,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
)

offset_distribution_figure.update_layout(
    title=dict(text="fixation offsets"),
    width=1000, height=500,
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=10, r=10, b=10, t=10, pad=0),
)
# offset_distribution_figure.layout.annotations = []   # remove subtitles

offset_distribution_figure.show()

### (3) Final Figure - save to file
Displaying only the $d'$ distributions for fixation onsets & offsets in the same figure, to avoid showing too many subplots

In [42]:
W, H = 600, 450

discriminability_figure = make_subplots(
    rows=2, cols=1, shared_xaxes=True, subplot_titles=['Fixation Onset', 'Fixation Offset'], vertical_spacing=0.1
)

# copy onset d-prime violins into new figure
for tr in onset_distribution_figure.data:
    if tr['scalegroup'] != 'd_prime':
        # ignore non d-prime violins
        continue
    new_tr = copy.deepcopy(tr)
    new_tr['width'] = 0.8   # make violins wider so there's less space between them
    discriminability_figure.add_trace(trace=new_tr, row=1, col=1)

# copy offset d-prime violins into new figure
for tr in offset_distribution_figure.data:
    if tr['scalegroup'] != 'd_prime':
        # ignore non d-prime violins
        continue
    new_tr = copy.deepcopy(tr)
    new_tr['width'] = 0.8   # make violins wider so there's less space between them
    discriminability_figure.add_trace(trace=new_tr, row=2, col=1)

discriminability_figure.update_layout(
    title=None,
    width=W, height=H,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    yaxis=dict(showgrid=False, zeroline=False, showline=False, tickfont=dict(size=14)),
    yaxis2=dict(showgrid=False, zeroline=False, showline=False, tickfont=dict(size=14)),
    xaxis2=dict(showgrid=False, tickfont=dict(size=14), tickangle=0),
    margin=dict(l=10, r=10, b=10, t=20, pad=0),
)
# discriminability_figure.layout.annotations = []   # remove subtitles


save_fig(discriminability_figure, FIG_ID, 'left', 'fixation-discriminability', IS_SUPP)
discriminability_figure